In [1]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re

#filename = "math sample for API testing.csv"
filename = "Big Math Subset.csv"

fields ={'Internal_ID': 'Internal_ID',
         'Strand': 'Strand',
         'Sub-Strand': 'Substrand',
         'Content Focus': 'Content_Focus',
         'Component': 'Component',
         'Number': 'Number',
         'Digits': 'Digits',
         'Shape': 'Shape',
         'Units': 'Units',
         'Steps': 'Steps',
         'Context': 'Context',
         'SIG Stem': 'SIG_Stem',
         'SIG Options': 'SIG_Options',
         'Additional Modifiers': 'Additional_Modifiers'}

In [2]:
def process_file(filename, fields, tagging_df):
    process_fields = fields.keys()
    data = []
    bad_data = []
    component_holding = []
#    data_keys = ['Internal ID', 'Strand','Substrand', 'Content_focus', 'Component', 'Number', 'Digits', 'Shape', 'Units', 'Steps', 'Context', 'SIG Stem', 'SIG Options', 'Additional Modifiers']
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for i in range(1):
            l = reader.next()

        for line in reader:
            error_test = False
            component_hold = False
            row = {}
            strand_dict = {}
            strand_list = []
            InternalID = int(line['Internal ID'])
            Strand = line['Strand'].split("; ")
            for strand in Strand:
                strand_sub_dict = {}
                content_focus_df = tagging_df['Content_Focus']
                StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
                strand_sub_dict = {'Strand': strand, 'StrandGUID' : StrandGUID, 'Substrand': []}
                strand_list.append(strand_sub_dict)
                         
            substrand_list = line['Sub-Strand'].split("; ")
            strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)
            
            content_focus_list = line['Content Focus'].split("; ")
            strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)
            
            Differentiator_Keys = ['Shape', 'Units', 'Steps']            
            Differentiator_List = []
            for Differentiator in Differentiator_Keys:
                Differentiator_values = line[Differentiator].split("; ")
                Differentiator_sub_list = []
                Differentiator_df = tagging_df[Differentiator]
                DifferentiatorGUID_Name = Differentiator + "GUID"
                for value in Differentiator_values:
                    Differentiator_sub_dict = {}
                    DifferentiatorGUID, error_test = get_GUID_lite(value, Differentiator_df, error_test)
                    Differentiator_sub_dict = {Differentiator: value, DifferentiatorGUID_Name : DifferentiatorGUID}
                    Differentiator_sub_list.append(Differentiator_sub_dict)
                Differentiator_List.append(Differentiator_sub_list)
            #Process Differentiators
            
            
            row = {"Internal_ID": InternalID, "Organizers" : strand_list, "Differentiators": Differentiator_List}
            if error_test == True:
                bad_data.append(row)
            elif component_hold == True:
                component_holding.append(row)
            else: data.append(row)
    #Create data frame from strand elements for data that passed
    #Infer additional matches based on subgroup
    #Process component_holding data
    
        
            
    return data, bad_data

In [60]:
data, bad_data = process_file(filename, fields, tagging_dict)
#pprint.pprint(bad_data)

In [61]:
len(bad_data)

130

In [62]:
pprint.pprint(data)

[{'Differentiators': [[{'Shape': 'N/A', 'ShapeGUID': 'N/A'}],
                      [{'Units': 'N/A', 'UnitsGUID': 'N/A'}],
                      [{'Steps': 'N/A', 'StepsGUID': 'N/A'}]],
  'Internal_ID': 329521,
  'Organizers': [{'Strand': 'Numbers and Operations',
                  'StrandGUID': 'D66EB2C0-8BFB-11E6-921B-89CC29C466BA',
                  'Substrand': [{'Content Focus': [{'Content Focus': 'Compare numbers',
                                                    'ContentFocusGUID': 'CBB39A90-8C1D-11E6-867F-EA7829C466BA'}],
                                 'Substrand': 'Numbers',
                                 'SubstrandGUID': '164F082A-8C17-11E6-81DC-467029C466BA'}]}]},
 {'Differentiators': [[{'Shape': 'Point',
                        'ShapeGUID': '1E24B42A-97C8-11E6-8048-1FDE29C466BA'}],
                      [{'Units': 'Generic',
                        'UnitsGUID': '7C0B3838-97C4-11E6-B4CA-B3D929C466BA'}],
                      [{'Steps': 'N/A', 'StepsGUID': 'N/A'}]],
 

In [3]:
def processSubstrands(strand_list, substrand_list, df, error_test):
    GUID = None
    for substrand in substrand_list:
        if len(df[(df['Title']==lowercase(substrand))]) == 0:  
            error_test = True
            error_message = 'Error: Substrand ' + substrand + ' not found in master list'
            strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'SubstrandGUID': error_message})
            break
    if len(substrand_list) < len(strand_list):
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: more strands than substrands'})
    elif len(strand_list) == 1: 
        for substrand in substrand_list:
            strand_list, error_test = getSubstrandGUID(strand_list[0]['Strand'], strand_list, substrand, df, error_test)    
    elif len(substrand_list) == 2 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 2)
    elif len(substrand_list) == 3 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 3)
    else: 
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: Multiple substrands, cannot parse yet'})
    return strand_list, error_test

def getSubstrandGUID(strand, strand_list, substrand, df, error_test): 
    if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
        SubstrandGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand)), 'Unique ID'].item()
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append({'Substrand': substrand, 'SubstrandGUID': SubstrandGUID, 'Content Focus': []})
    else: 
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append({'Substrand': substrand, 'SubstrandGUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

    

In [ ]:
strand_raw = "Geometry and Measurement; Algebra"
substrand_raw = "Shapes; Functions"
content_focus_raw = "Use unit circle; Know trigonometric ratios"
#print strand_raw
#print substrand_raw

error_test = False
strand_dict = {}
strand_list = []
temp_list = strand_raw.split("; ")
#print temp_list
for strand in temp_list:
    strand_sub_dict = {}
    #content_focus_df = tagging_df['Content_Focus']
    StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
    strand_sub_dict = {'Strand': strand, 'StrandGUID' : StrandGUID, 'Substrand': []}
    strand_list.append(strand_sub_dict)

substrand_list = substrand_raw.split("; ")
strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)

content_focus_list = content_focus_raw.split('; ')
#strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)


In [59]:
def processContentFocus(strand_list, substrand_list, content_focus_list, df, error_test):
    GUID = None
    for content_focus in content_focus_list:
        if len(df[(df['Title']==lowercase(content_focus))]) == 0:  
            error_test = True
            error_message = 'Error: Content Focus ' + content_focus + ' not found in master list'
            strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'ContentFocusGUID': error_message})
            break
    if len(content_focus_list) < len(substrand_list):
        error_test = True
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'ContentFocusGUID': 'Error: more Substrands than Content Focus terms'})
    elif len(substrand_list) == 1: 
        for content_focus in content_focus_list:
            strand_list, error_test = getContentFocusGUID(strand_list[0]['Strand'], strand_list, substrand_list[0], content_focus, df, error_test)    
    elif len(substrand_list) == 2 and len(content_focus_list) == 2:
        strand_list, error_test = matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, 2)
    elif len(substrand_list) == 3 and len(content_focus_list) == 3:
        strand_list, error_test = matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, 3)
    else: 
        error_test = True
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'ContentFocusGUID': 'Error: Multiple content Focus terms, cannot parse yet'})
    return strand_list, error_test

In [40]:
def processInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, component_hold):
    GUID = None
    for component in component_list:
        if len(df[(df['Title']==lowercase(component))]) == 0:  
            error_test = True
            error_message = 'Error: Component ' + component + ' not found in master list'
            strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, 'ComponentGUID': error_message})
            break
    if len(content_focus_list) == 1:
        for component in component_list:
            strand_list, error_test, component_hold = getComponentGUID(strand_list[0]['Strand'], strand_list, substrand_list[0], content_focus_list[0], component, df, error_test, component_hold) 
    elif len(substrand_list) == len(component_list):
        strand_list, error_test, component_hold = matchInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, len(component_list))            
    else: 
        component_hold = True
        strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, 'ComponentGUID': 'Component Hold'})
    return strand_list, error_test, component_hold

In [41]:
def getComponentGUID(strand, strand_list, substrand, content_focus, component, df, error_test, component_hold): 
    strand_index = next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)
    substrand_index = next(index for (index, d) in enumerate(strand_list[strand_index]['Substrand']) if d["Substrand"] == substrand)    
    content_focus_index = next(index for (index, d) in enumerate(strand_list[strand_index]['Substrand'][substrand_index]['Content Focus']) if d["Content Focus"] == content_focus)
    if len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append({'Component': Component, 'ComponentGUID': ComponentGUID})
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Example Content Focus']==lowercase(content_focus)) & (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append({'Component': Component, 'ComponentGUID': ComponentGUID})
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component))]) > 1:
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append({'Component': component, 'ComponentGUID': 'Holding'})
        component_hold = True
    else: 
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append({'Component': component, 'ComponentGUID': 'Error: No unique Strand/Substrand/Component match found in master list'})
        error_test = True
    return strand_list, error_test, component_hold

In [42]:
def matchInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, matchlimit):
    strand_substrand_names =[]
    for strand in strand_list:
        strand_name = strand['Strand']
        for substrand in strand['Substrand']:
            strand_substrand_names.append(strand_name + ', ' + substrand['Substrand'])

    match_status = pd.DataFrame(False,index=strand_substrand_names,columns=component_list)
    print match_status
    strand_substrand_names[0].split(', ')[0]

    for strand_substrand in strand_substrand_names:
        #Need to use substrand and also pull strand from strand list, might be easier to build from strand, down
        for component in component_list:
            if len(df[(df['Strand']==lowercase(strand_substrand.split(', ')[0])) & (df['Sub-Strand']==lowercase(strand_substrand.split(', ')[1]))& (df['Title']==lowercase(content_focus))]) == 1:
                match_status.ix[strand_substrand, content_focus] = True

    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
            strand_list, error_test = getComponentGUID(pair[0].split(', ')[0], strand_list, pair[0].split(', ')[1], pair[1], df, error_test)
    else: 
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'ContentFocusGUID': 'Error: Ambiguous strand/substrand/content focus relationships'})
        error_test = True
    return strand_list, error_test

In [43]:
strand = 'Geometry and Measurement'
substrand_list = ['Shapes']
content_focus_list = ['classify shapes', 'compose shapes']
component_list = ['Sides']
error_test = False
component_hold = False
#strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)

strand_list=[]
strand_sub_dict = {}
StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
strand_sub_dict = {'Strand': strand, 'StrandGUID' : StrandGUID, 'Substrand': []}
strand_list.append(strand_sub_dict)
                         
strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)

strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)
strand_list, error_test, component_hold = processInitialComponent(strand_list, substrand_list, content_focus_list, component_list, content_focus_df, error_test, component_hold)

pprint.pprint(strand_list[0])

KeyError: 'Component'

In [5]:
def getContentFocusGUID(strand, strand_list, substrand, content_focus, df, error_test): 
    strand_index = next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)
    substrand_index = next(index for (index, d) in enumerate(strand_list[strand_index]['Substrand']) if d["Substrand"] == substrand)    
    if len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(content_focus))]) == 1:
        ContentFocusGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(content_focus)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'].append({'Content Focus': content_focus, 'ContentFocusGUID': ContentFocusGUID})
    else: 
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'].append({'Content Focus': content_focus, 'ContentFocusGUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

In [21]:
def matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, matchlimit):
    strand_substrand_names =[]
    for strand in strand_list:
        strand_name = strand['Strand']
        for substrand in strand['Substrand']:
            strand_substrand_names.append(strand_name + ', ' + substrand['Substrand'])

    match_status = pd.DataFrame(False,index=strand_substrand_names,columns=content_focus_list)

    strand_substrand_names[0].split(', ')[0]

    for strand_substrand in strand_substrand_names:
        #Need to use substrand and also pull strand from strand list, might be easier to build from strand, down
        for content_focus in content_focus_list:
            if len(df[(df['Strand']==lowercase(strand_substrand.split(', ')[0])) & (df['Sub-Strand']==lowercase(strand_substrand.split(', ')[1]))& (df['Title']==lowercase(content_focus))]) == 1:
                match_status.ix[strand_substrand, content_focus] = True

    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
            strand_list, error_test = getContentFocusGUID(pair[0].split(', ')[0], strand_list, pair[0].split(', ')[1], pair[1], df, error_test)
    else: 
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'ContentFocusGUID': 'Error: Ambiguous strand/substrand/content focus relationships'})
        error_test = True
    return strand_list, error_test

In [7]:
#Build a data frame to track match status to determine whether strand-substrand pairings can be deduced    
def matchStrandsSubstrands(strand_list, substrand_list, df, error_test, matchlimit):
    strand_names = []
    for strand in strand_list:
        strand_names.append(strand['Strand'])

    match_status = pd.DataFrame(False,index=strand_names,columns=substrand_list)

    for strand in strand_names:
        for substrand in substrand_list:
            if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
                match_status.ix[strand,substrand] = True
    
    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        for pair in true_pairs:
            strand_list, error_test = getSubstrandGUID(pair[0], strand_list, pair[1], df, error_test)
    else: 
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: Ambiguous strand/substrand relationships'})
        error_test = True
    return strand_list, error_test
       

In [8]:
def lowercase(text):
    return unicode(text, "latin2").lower()

In [9]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    df_lowercase(data, 'Title')
    if len(data.columns) == 6:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
    if len(data.columns) == 7:
        df_lowercase(data, 'Strand')
    return data

In [10]:
def df_lowercase(dataframe, column):
    lowercase = lambda x: unicode(x, "latin2").lower()
    dataframe[column] = dataframe[column].apply(lowercase)
    return dataframe

In [29]:
steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
number_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")
content_focus_df = Load_GUIDs("Math_Content_Foc.csv")
component_df = Load_GUIDs("Math_Components.csv")
tagging_dict = {'Steps': steps_df, 'Shape': shapes_df, 'Number': number_df, 'Units': units_df, 'Content_Focus': content_focus_df, 'Component': component_df}

In [12]:

def get_GUID_lite(tag, df, error_test): 
    #get GUID for steps, number, units, shape
    tag = unicode(tag, "latin2").lower()
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        if tag == r'n/a':
            GUID = r'N/A'
        else: 
            GUID = "Error: no matches found"
            error_test = True
    else: 
        GUID = "Error: resolve multiple values"
        error_test = True
    return GUID, error_test

In [ ]:
number_test = "Square roots"
error_test = False
print get_GUID_lite(number_test, number_df, error_test)

In [ ]:

#create version for content focus and component
#Play around with passing multiple conditions into first .ix argument

tag = 'N/A'
if tag == r'N/A':
    print tag


In [ ]:
'''
Data structure

{Internal ID: value
    {Organizers: [
        {Strand: [ {Strand: Strand_value,
                   Strand_GUID: GUID,
                   Substrand: [{Substrand: Substrand_value,
                                Substrand_GUID: GUID,
                                Content_Focus: [{Content_Focus: Content_Focus_value,                       
                                                Content_Focus_GUID: GUID,
                                                Component: [{Component: Component_value,
                                                              Component_GUID: GUID}]}]}]}]
        {Descriptors: {Number: [N1, N2, ...]}
                        {Digits: [D1, D2, ...]}
            {Shape: [Shape1, ...]}
            {Units: [Units1, ...]}
            {Steps: Steps}
            {Context: Context}
            {SIG Stem: [Sig Stem1, ...]}
            {SIG Options: [SIG Options1, ...]}
            {Additional Modifiers: [AM1, ...]}
}

'''